In [ ]:
import pandas as pd
# PunktSentenceTokenizer
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import naive_bayes
from sklearn.model_selection import cross_val_score
import librosa

from librosa.feature import melspectrogram

In [ ]:
def prepare_song(song_path):
  y,sr = librosa.load(song_path,sr=10000)
  return librosa.feature.melspectrogram(y=y, sr=sr)
  

In [ ]:
all_tracks = [None] * 13936
sex = [None] * 13936
i = 0
import csv
with open('C:\\Users\\Xiaomi\\Desktop\\ML-biometry-task\\train\\targets.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    for song_name in tsv_file:
        song_pieces = prepare_song('C:\\Users\\Xiaomi\\Desktop\\ML-biometry-task\\train\\' + song_name[0] + '.wav')
        all_tracks[i] =  song_pieces
        sex[i] = song_name[1]
        i += 1
        if i % 100 == 0:
            print(i)

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 196, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

model.summary()